In [1]:
from pathlib import Path
import pandas as pd
import numpy as np

In [2]:
from utils import generate_table, Cell, simple_to_latex

In [3]:
data = pd.read_csv("../../results/cmnist/2_digits/2colors/30seeds_base_post_iclr_submission.csv")

In [4]:
data["Group"].value_counts()

ranking-fdm.cmnist-2digits-2colors-missing_s-baseline                30
kmeans-fdm.cmnist-2digits-2colors-mildy_subsampled-oldencoder        30
no-cluster-fdm.cmnist.2digits.2colors.mildly_subsampled.baseline     30
perfect-cluster.cmnist.2digits.2colors.mildly_subsampled.baseline    30
kmeans-fdm/cmnist/2digits/2colors/missing_s/baseline                 30
/cmnist/2digits/2colors/missing_s/baseline                           30
perfect-cluster/cmnist/2digits/2colors/missing_s/baseline            30
ranking-fdm.cmnist-2digits-2colors-mildly_sumbsampled-baseline       30
kmeans-fdm.cmnist-2digits-2colors-missing_s-oldencoder               30
Name: Group, dtype: int64

In [5]:
data = data.replace({"Group": {
    "kmeans-fdm/cmnist/2digits/2colors/missing_s/baseline": r"k-means (new encoder)",
    "perfect-cluster.cmnist.2digits.2colors.mildly_subsampled.baseline": r"\texttt{ZSF+bal.\ (ground truth)}",
    "ranking-fdm.cmnist-2digits-2colors-missing_s-baseline": r"\texttt{ZSF+bal.\ (ranking)}",
    "kmeans-fdm.cmnist-2digits-2colors-mildy_subsampled-oldencoder": r"\texttt{ZSF+bal.\ (k-means)}",
    "/cmnist/2digits/2colors/missing_s/baseline": r"\texttt{ZSF}",
    "kmeans-fdm.cmnist-2digits-2colors-missing_s-oldencoder": r"\texttt{ZSF+bal.\ (k-means)}",
    "ranking-fdm.cmnist-2digits-2colors-mildly_sumbsampled-baseline": r"\texttt{ZSF+bal.\ (ranking)}",
    "perfect-cluster/cmnist/2digits/2colors/missing_s/baseline": r"\texttt{ZSF+bal.\ (ground truth)}",
    "no-cluster-fdm.cmnist.2digits.2colors.mildly_subsampled.baseline": r"\texttt{ZSF}",
}}, inplace=False)

In [6]:
def merge(a, b):
    data[a] = data[a].combine_first(data[b])

merge("TPR_colour_0.0÷colour_1.0 (pytorch_classifier)", "TPR_colour_0.0/colour_1.0 (pytorch_classifier)")
merge("TNR_colour_0.0÷colour_1.0 (pytorch_classifier)", "TNR_colour_0.0/colour_1.0 (pytorch_classifier)")
merge('prob_pos_colour_0.0÷colour_1.0 (pytorch_classifier)', 'prob_pos_colour_0.0/colour_1.0 (pytorch_classifier)')

In [7]:
data.columns

Index(['Name', 'User', 'Created', 'Runtime', 'State', 'Notes', 'Tags', 'Group',
       'dataset', 'filter_labels', 'balanced_context', 'disc_weight',
       'elbo_weight', 'enc_levels', 'lr', 'num_disc_updates', 'num_discs',
       'seed', 'subsample_context.0', 'subsample_context.1',
       'subsample_context.2', 'subsample_context.3', 'subsample_train.0',
       'subsample_train.1', 'subsample_train.2', 'subsample_train.3',
       'zs_frac', 'enc_init_chans', 'enc_out_dim', 'kl_weight', 'oversample',
       'disc_method', 'Accuracy (pytorch_classifier)',
       'Clust/Context Accuracy', 'Clust/Context ARI',
       'Clust/Context Acc y=0 s=0', 'Clust/Context Acc y=0 s=1',
       'Clust/Context Acc y=1 s=0', 'Clust/Context Acc y=1 s=1',
       'Clust/Context NMI', 'Clust/Test ARI', 'Clust/Test Acc y=0 s=0',
       'Clust/Test Acc y=0 s=1', 'Clust/Test Acc y=1 s=0',
       'Clust/Test Acc y=1 s=1', 'Clust/Test Accuracy', 'Clust/Test NMI',
       'Renyi preds and s (pytorch_classifier)',

In [14]:
def table(filter_, cols, groupby: str = "Group"):
    return (
        data
        .query(filter_)
        .groupby(groupby, sort=False)
        .agg(Cell(round_to=3))[list(cols)]
        .reset_index(level=[groupby], inplace=False)
        .rename(columns={**cols, 'Group': "Method"}, inplace=False)
    )

In [26]:
cols = {
    'Clust/Context Accuracy': "Clust Acc",
#     'Clust/Context NMI': "Clust NMI",
#     'Clust/Context ARI': "Clust ARI",
    "Accuracy (pytorch_classifier)": "Accuracy",
    'prob_pos_colour_0.0÷colour_1.0 (pytorch_classifier)': "AR ratio",
#     "Renyi preds and s (pytorch_classifier)": "Renyi corr",
    "TPR_colour_0.0÷colour_1.0 (pytorch_classifier)": "TPR ratio",
    'TNR_colour_0.0÷colour_1.0 (pytorch_classifier)': "TNR ratio",
}

In [27]:
res_partial = table("`subsample_train.0` == 1.0", cols)
res_partial

,Method,Clust Acc,Accuracy,AR ratio,TPR ratio,TNR ratio
0,\texttt{ZSF},N/A,0.829 $\pm$ 0.083,0.306 $\pm$ 0.337,0.31 $\pm$ 0.342,0.994 $\pm$ 0.006
1,\texttt{ZSF+bal.\ (ground truth)},N/A,0.92 $\pm$ 0.058,0.686 $\pm$ 0.245,0.684 $\pm$ 0.241,0.997 $\pm$ 0.003
2,\texttt{ZSF+bal.\ (ranking)},0.983 $\pm$ 0.006,0.912 $\pm$ 0.056,0.653 $\pm$ 0.232,0.652 $\pm$ 0.227,0.995 $\pm$ 0.005
3,\texttt{ZSF+bal.\ (k-means)},0.732 $\pm$ 0.157,0.904 $\pm$ 0.076,0.627 $\pm$ 0.308,0.63 $\pm$ 0.313,0.995 $\pm$ 0.004


In [28]:
res_miss_s = table("`subsample_train.0` == 0.0", cols)
res_miss_s

,Method,Clust Acc,Accuracy,AR ratio,TPR ratio,TNR ratio
0,\texttt{ZSF+bal.\ (ranking)},0.922 $\pm$ 0.024,0.868 $\pm$ 0.114,0.876 $\pm$ 0.097,0.783 $\pm$ 0.265,0.702 $\pm$ 0.23
1,\texttt{ZSF+bal.\ (k-means)},0.716 $\pm$ 0.153,0.796 $\pm$ 0.104,0.688 $\pm$ 0.219,0.709 $\pm$ 0.328,0.492 $\pm$ 0.296
2,\texttt{ZSF+bal.\ (ground truth)},N/A,0.888 $\pm$ 0.088,0.854 $\pm$ 0.089,0.852 $\pm$ 0.175,0.714 $\pm$ 0.203
3,\texttt{ZSF},N/A,0.851 $\pm$ 0.143,0.729 $\pm$ 0.257,0.622 $\pm$ 0.405,0.797 $\pm$ 0.261
4,k-means (new encoder),N/A,0.81 $\pm$ 0.132,0.751 $\pm$ 0.165,0.712 $\pm$ 0.345,0.541 $\pm$ 0.308


In [29]:
# baseline files
baseline_files = {
    r"Kamiran \& Calders (2012) CNN": "cmnist_cnn_baseline_color_60epochs.csv",
    r"\texttt{FWD \cite{HasSriNamLia18}}": "cmnist_dro_baseline_color_60epochs.csv",
}

In [30]:
base_path = "../../results/cmnist/2_digits/2colors"
def collate(file_dict, dir_):
    df_all = pd.DataFrame()
    for log_method, filename in file_dict.items():
        df = pd.read_csv(Path(base_path) / dir_ / filename)
#         df.insert(0, "exp_group", exp_group)
        df.insert(0, "log_method", log_method)
        if log_method == r"\texttt{FWD \cite{HasSriNamLia18}}":
            df["log_method"] = log_method + " (" + df["eta"].astype(str) + ")"
        df_all = pd.concat([df_all, df], axis="index", ignore_index=True, sort=False)
    return df_all

In [31]:
miss_s = collate(baseline_files, "miss_s")
partial = collate(baseline_files, "partial")

In [32]:
bl_cols = {
#     'Clust/Context Accuracy': "Clust Acc",
#     'Clust/Context NMI': "Clust NMI",
#     'Clust/Context ARI': "Clust ARI",
    "Accuracy": "Accuracy",
    'prob_pos_colour_0.0÷colour_1.0': "AR ratio",
#     "Renyi preds and s": "Renyi corr",
    "TPR_colour_0.0÷colour_1.0": "TPR ratio",
    'TNR_colour_0.0÷colour_1.0': "TNR ratio",
}

In [33]:
groupby = "log_method"
res2_partial = (partial[[groupby] + list(bl_cols)]
        .groupby([groupby], sort=False)
        .agg(Cell(round_to=3)).rename(columns=bl_cols, inplace=False)
        .reset_index(level=[groupby], inplace=False)
        .rename(columns={groupby: "Method"}, inplace=False)
       )
res2_partial

,Method,Accuracy,AR ratio,TPR ratio,TNR ratio
0,Kamiran \& Calders (2012) CNN,0.756 $\pm$ 0.009,0.001 $\pm$ 0.005,0.001 $\pm$ 0.005,0.994 $\pm$ 0.004
1,\texttt{FWD \cite{HasSriNamLia18}} (0.01),0.757 $\pm$ 0.009,0.008 $\pm$ 0.018,0.008 $\pm$ 0.018,0.994 $\pm$ 0.005
2,\texttt{FWD \cite{HasSriNamLia18}} (0.1),0.762 $\pm$ 0.016,0.026 $\pm$ 0.06,0.026 $\pm$ 0.059,0.994 $\pm$ 0.004
3,\texttt{FWD \cite{HasSriNamLia18}} (0.3),0.765 $\pm$ 0.026,0.045 $\pm$ 0.106,0.045 $\pm$ 0.104,0.989 $\pm$ 0.027
4,\texttt{FWD \cite{HasSriNamLia18}} (1.0),0.576 $\pm$ 0.09,0.003 $\pm$ 0.013,0.006 $\pm$ 0.026,0.593 $\pm$ 0.442


In [34]:
groupby = "log_method"
res2_miss_s = (miss_s[[groupby] + list(bl_cols)]
        .groupby([groupby], sort=False)
        .agg(Cell(round_to=3)).rename(columns=bl_cols, inplace=False)
        .reset_index(level=[groupby], inplace=False)
        .rename(columns={groupby: "Method"}, inplace=False)
       )
res2_miss_s

,Method,Accuracy,AR ratio,TPR ratio,TNR ratio
0,Kamiran \& Calders (2012) CNN,0.759 $\pm$ 0.04,0.239 $\pm$ 0.304,0.288 $\pm$ 0.398,0.738 $\pm$ 0.376
1,\texttt{FWD \cite{HasSriNamLia18}} (0.01),0.77 $\pm$ 0.057,0.226 $\pm$ 0.319,0.224 $\pm$ 0.358,0.846 $\pm$ 0.298
2,\texttt{FWD \cite{HasSriNamLia18}} (0.1),0.772 $\pm$ 0.062,0.264 $\pm$ 0.337,0.303 $\pm$ 0.419,0.781 $\pm$ 0.338
3,\texttt{FWD \cite{HasSriNamLia18}} (0.3),0.763 $\pm$ 0.041,0.355 $\pm$ 0.302,0.525 $\pm$ 0.454,0.531 $\pm$ 0.432
4,\texttt{FWD \cite{HasSriNamLia18}} (1.0),0.687 $\pm$ 0.07,0.306 $\pm$ 0.188,0.602 $\pm$ 0.387,0.171 $\pm$ 0.348


In [35]:
table_partial = pd.concat([res_partial, res2_partial], sort=False)
table_partial

,Method,Clust Acc,Accuracy,AR ratio,TPR ratio,TNR ratio
0,\texttt{ZSF},N/A,0.829 $\pm$ 0.083,0.306 $\pm$ 0.337,0.31 $\pm$ 0.342,0.994 $\pm$ 0.006
1,\texttt{ZSF+bal.\ (ground truth)},N/A,0.92 $\pm$ 0.058,0.686 $\pm$ 0.245,0.684 $\pm$ 0.241,0.997 $\pm$ 0.003
2,\texttt{ZSF+bal.\ (ranking)},0.983 $\pm$ 0.006,0.912 $\pm$ 0.056,0.653 $\pm$ 0.232,0.652 $\pm$ 0.227,0.995 $\pm$ 0.005
3,\texttt{ZSF+bal.\ (k-means)},0.732 $\pm$ 0.157,0.904 $\pm$ 0.076,0.627 $\pm$ 0.308,0.63 $\pm$ 0.313,0.995 $\pm$ 0.004
0,Kamiran \& Calders (2012) CNN,NaN,0.756 $\pm$ 0.009,0.001 $\pm$ 0.005,0.001 $\pm$ 0.005,0.994 $\pm$ 0.004
1,\texttt{FWD \cite{HasSriNamLia18}} (0.01),NaN,0.757 $\pm$ 0.009,0.008 $\pm$ 0.018,0.008 $\pm$ 0.018,0.994 $\pm$ 0.005
2,\texttt{FWD \cite{HasSriNamLia18}} (0.1),NaN,0.762 $\pm$ 0.016,0.026 $\pm$ 0.06,0.026 $\pm$ 0.059,0.994 $\pm$ 0.004
3,\texttt{FWD \cite{HasSriNamLia18}} (0.3),NaN,0.765 $\pm$ 0.026,0.045 $\pm$ 0.106,0.045 $\pm$ 0.104,0.989 $\pm$ 0.027
4,\texttt{FWD \cite{HasSriNamLia18}} (1.0),NaN,0.576 $\pm$ 0.09,0.003 $\pm$ 0.013,0.006 $\pm$ 0.026,0.593 $\pm$ 0.442


In [37]:
simple_to_latex(table_partial)

\begin{tabular}{llllll}
\toprule
                                    Method &          Clust Acc &           Accuracy &           AR ratio &          TPR ratio &          TNR ratio \\
\midrule
                              \texttt{ZSF} &                N/A &  0.829 $\pm$ 0.083 &  0.306 $\pm$ 0.337 &   0.31 $\pm$ 0.342 &  0.994 $\pm$ 0.006 \\
         \texttt{ZSF+bal.\ (ground truth)} &                N/A &   0.92 $\pm$ 0.058 &  0.686 $\pm$ 0.245 &  0.684 $\pm$ 0.241 &  0.997 $\pm$ 0.003 \\
              \texttt{ZSF+bal.\ (ranking)} &  0.983 $\pm$ 0.006 &  0.912 $\pm$ 0.056 &  0.653 $\pm$ 0.232 &  0.652 $\pm$ 0.227 &  0.995 $\pm$ 0.005 \\
              \texttt{ZSF+bal.\ (k-means)} &  0.732 $\pm$ 0.157 &  0.904 $\pm$ 0.076 &  0.627 $\pm$ 0.308 &   0.63 $\pm$ 0.313 &  0.995 $\pm$ 0.004 \\
             Kamiran \& Calders (2012) CNN &                NaN &  0.756 $\pm$ 0.009 &  0.001 $\pm$ 0.005 &  0.001 $\pm$ 0.005 &  0.994 $\pm$ 0.004 \\
 \texttt{FWD \cite{HasSriNamLia18}} (0.01) &        

In [36]:
table_miss_s = pd.concat([res_miss_s, res2_miss_s], sort=False)
table_miss_s

,Method,Clust Acc,Accuracy,AR ratio,TPR ratio,TNR ratio
0,\texttt{ZSF+bal.\ (ranking)},0.922 $\pm$ 0.024,0.868 $\pm$ 0.114,0.876 $\pm$ 0.097,0.783 $\pm$ 0.265,0.702 $\pm$ 0.23
1,\texttt{ZSF+bal.\ (k-means)},0.716 $\pm$ 0.153,0.796 $\pm$ 0.104,0.688 $\pm$ 0.219,0.709 $\pm$ 0.328,0.492 $\pm$ 0.296
2,\texttt{ZSF+bal.\ (ground truth)},N/A,0.888 $\pm$ 0.088,0.854 $\pm$ 0.089,0.852 $\pm$ 0.175,0.714 $\pm$ 0.203
3,\texttt{ZSF},N/A,0.851 $\pm$ 0.143,0.729 $\pm$ 0.257,0.622 $\pm$ 0.405,0.797 $\pm$ 0.261
4,k-means (new encoder),N/A,0.81 $\pm$ 0.132,0.751 $\pm$ 0.165,0.712 $\pm$ 0.345,0.541 $\pm$ 0.308
0,Kamiran \& Calders (2012) CNN,NaN,0.759 $\pm$ 0.04,0.239 $\pm$ 0.304,0.288 $\pm$ 0.398,0.738 $\pm$ 0.376
1,\texttt{FWD \cite{HasSriNamLia18}} (0.01),NaN,0.77 $\pm$ 0.057,0.226 $\pm$ 0.319,0.224 $\pm$ 0.358,0.846 $\pm$ 0.298
2,\texttt{FWD \cite{HasSriNamLia18}} (0.1),NaN,0.772 $\pm$ 0.062,0.264 $\pm$ 0.337,0.303 $\pm$ 0.419,0.781 $\pm$ 0.338
3,\texttt{FWD \cite{HasSriNamLia18}} (0.3),NaN,0.763 $\pm$ 0.041,0.355 $\pm$ 0.302,0.525 $\pm$ 0.454,0.531 $\pm$ 0.432
4,\texttt{FWD \cite{HasSriNamLia18}} (1.0),NaN,0.687 $\pm$ 0.07,0.306 $\pm$ 0.188,0.602 $\pm$ 0.387,0.171 $\pm$ 0.348


In [38]:
simple_to_latex(table_miss_s)

\begin{tabular}{llllll}
\toprule
                                    Method &          Clust Acc &           Accuracy &           AR ratio &          TPR ratio &          TNR ratio \\
\midrule
              \texttt{ZSF+bal.\ (ranking)} &  0.922 $\pm$ 0.024 &  0.868 $\pm$ 0.114 &  0.876 $\pm$ 0.097 &  0.783 $\pm$ 0.265 &   0.702 $\pm$ 0.23 \\
              \texttt{ZSF+bal.\ (k-means)} &  0.716 $\pm$ 0.153 &  0.796 $\pm$ 0.104 &  0.688 $\pm$ 0.219 &  0.709 $\pm$ 0.328 &  0.492 $\pm$ 0.296 \\
         \texttt{ZSF+bal.\ (ground truth)} &                N/A &  0.888 $\pm$ 0.088 &  0.854 $\pm$ 0.089 &  0.852 $\pm$ 0.175 &  0.714 $\pm$ 0.203 \\
                              \texttt{ZSF} &                N/A &  0.851 $\pm$ 0.143 &  0.729 $\pm$ 0.257 &  0.622 $\pm$ 0.405 &  0.797 $\pm$ 0.261 \\
                     k-means (new encoder) &                N/A &   0.81 $\pm$ 0.132 &  0.751 $\pm$ 0.165 &  0.712 $\pm$ 0.345 &  0.541 $\pm$ 0.308 \\
             Kamiran \& Calders (2012) CNN &        